In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os
from pathlib import Path

import numpy as np
from tifffile import imread
import matplotlib.pyplot as plt

from pasnascope import roi, vnc_length, initial_mask

In [ ]:
project_dir = Path(os.getcwd()).parent
img_dir = os.path.join(project_dir, 'data', 'embryos')

measurements = [f for f in sorted(os.listdir(img_dir)) if f.endswith('.csv')]
print(measurements)

movies = [f for f in sorted(os.listdir(img_dir)) if f.endswith('ch2.tif')]

## Comparing manual measurements and calculated values

The manually measured values were obtained using ImageJ, and exported as a CSV file.

The calculated values use the `pasnascope` module.

For comparing manual measurements and calculated values, make sure that both have the same sampling interval. 

In [ ]:
def get_vnc_length_from_csv(file_path, interval=20):
    '''Reads csv data as a nparray.

    The csv data contains the manual measurements extracted with ImageJ.
    Column 0 contains indices and column 7 contains Feret's diameter.'''
    data = np.genfromtxt(
        file_path, delimiter=',', skip_header=1, usecols=(0,7))
    indices = data[:, 0]
    indices -= 1
    indices *= interval
    return data

def get_vnc_length_from_tiff(file_path, interval=20, pixel_width=1.62, mask=False, orientation='v'):
    '''Measures VNC length using the `pasnascope` module.'''
    img = imread(file_path)
    img = img[::interval,:,:]
    if mask:
        initial_mask = roi.get_initial_mask(img, 200)
    else:
        initial_mask = None
    img_roi = roi.get_roi(img, window=1, mask=initial_mask, orientation=orientation)
    return vnc_length.measure_VNC(img_roi)*pixel_width

def get_vnc_length_ell_fit(file_path, interval=20, pixel_width=1.62):
    '''Measures VNC length, first masks the image with an ellipse.'''
    img = imread(file_path)
    img = img[::interval,:,:]
    ell = initial_mask.fit_ellipse(img)
    mask = initial_mask.create_mask_from_ellipse(ell, img[0].shape)
    img_roi = roi.get_roi(img, window=1, mask=mask, orientation='v')
    return vnc_length.measure_VNC(img_roi)*pixel_width


In [ ]:
sampling_interval = 20
PIXEL_WIDTH = 1.62

for csv_file, tiff_file in zip(measurements, movies):
    fig, ax = plt.subplots()
    fig.canvas.header_visible = False
    fig.canvas.resizable = False
    fig.suptitle(tiff_file[:-8])

    manual = get_vnc_length_from_csv(os.path.join(img_dir, csv_file))
    calc = get_vnc_length_from_tiff(os.path.join(img_dir, tiff_file))

    x = manual[:, 0]
    y = manual[:, 1]

    ax.plot(x, y, label='manual')
    ax.plot(x, calc, label='calculated')
    ax.legend()


Comparisons for embryos in ventral orientation (single otsu threshold and no mask):

In [ ]:
v_orientation = ['emb11-ch2.tif', 'emb12-ch2.tif', 'emb13-ch2.tif', 'emb31-ch2.tif', 'emb52-ch2.tif', 'emb63-ch2.tif']
v_measurements = [f.split('.')[0] + '-lengths.csv' for f in v_orientation]

sampling_interval = 20
PIXEL_WIDTH = 1.62

errors = []
for csv_file, tiff_file in zip(v_measurements, v_orientation, strict=True):
    manual = get_vnc_length_from_csv(os.path.join(img_dir, csv_file))
    calc = get_vnc_length_from_tiff(os.path.join(img_dir, tiff_file))

    x = manual[:, 0]
    y = manual[:, 1]

    err = (calc - y)/y
    errors.append((np.average(err), np.std(err)))

fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
fig.suptitle('Ventral orientation')

embryo_names = [m[:-8] for m in v_orientation]
avgs, yerr = zip(*errors)

ax.errorbar(embryo_names, avgs, yerr);

Comparisons for embryos in lateral position (3 classes otsu and mask)

In [ ]:
l_orientation = ['emb33-ch2.tif', 'emb43-ch2.tif', 'emb51-ch2.tif']
l_measurements = [f.split('.')[0] + '-lengths.csv' for f in l_orientation]

sampling_interval = 20
PIXEL_WIDTH = 1.62

errors = []
for csv_file, tiff_file in zip(l_measurements, l_orientation, strict=True):
    manual = get_vnc_length_from_csv(os.path.join(img_dir, csv_file))
    calc = get_vnc_length_from_tiff(os.path.join(img_dir, tiff_file), mask=True, orientation='l')

    x = manual[:, 0]
    y = manual[:, 1]

    err = (calc - y)/y
    errors.append((np.average(err), np.std(err)))

fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
fig.suptitle('Lateral orientation - masked from higher threshold')

embryo_names = [m[:-8] for m in l_orientation]
avgs, yerr = zip(*errors)

ax.errorbar(embryo_names, avgs, yerr);

In [ ]:
l_orientation = ['emb33-ch2.tif', 'emb43-ch2.tif', 'emb51-ch2.tif']
l_measurements = [f.split('.')[0] + '-lengths.csv' for f in l_orientation]

sampling_interval = 20
PIXEL_WIDTH = 1.62

errors = []
for csv_file, tiff_file in zip(l_measurements, l_orientation, strict=True):
    manual = get_vnc_length_from_csv(os.path.join(img_dir, csv_file))
    calc = get_vnc_length_ell_fit(os.path.join(img_dir, tiff_file))

    x = manual[:, 0]
    y = manual[:, 1]

    err = (calc - y)/y
    errors.append((np.average(err), np.std(err)))

fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
fig.suptitle('Lateral orientation - ellipse')

embryo_names = [m[:-8] for m in l_orientation]
avgs, yerr = zip(*errors)

ax.errorbar(embryo_names, avgs, yerr);